Tensor & Data Representation

In [1]:
import tensorflow as tf
import numpy as np

# Tensor dasar
t1 = tf.constant([[1, 2, 3], [4, 5, 6]])
t2 = tf.convert_to_tensor(np.array([7, 8, 9]))

print(t1)
print(t2)

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
tf.Tensor([7 8 9], shape=(3,), dtype=int32)


Ragged Tensor

In [3]:
ragged = tf.ragged.constant([
    [1, 2, 3],
    [4, 5],
    [6]
])
print(ragged)

sparse = tf.sparse.SparseTensor(
    indices=[[0, 0], [1, 2]],
    values=[10, 20],
    dense_shape=[3, 4]
)
print(tf.sparse.to_dense(sparse))


<tf.RaggedTensor [[1, 2, 3], [4, 5], [6]]>
tf.Tensor(
[[10  0  0  0]
 [ 0  0 20  0]
 [ 0  0  0  0]], shape=(3, 4), dtype=int32)


Variables & Trainable Parameters

In [4]:
w = tf.Variable([[1.0, 2.0], [3.0, 4.0]])
b = tf.Variable([1.0, 1.0])

# Update nilai
w.assign(w * 2)
b.assign_add([0.5, 0.5])

print(w)
print(b)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[2., 4.],
       [6., 8.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.5, 1.5], dtype=float32)>


Custom Layer

In [5]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
layer = MyDense(3)
x = tf.random.normal((2, 4))
y = layer(x)
print(y)



tf.Tensor(
[[ 1.976232   -2.3936684   0.26496792]
 [-0.3307743  -0.12615761 -1.3565416 ]], shape=(2, 3), dtype=float32)


Custom Model

In [6]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = MyDense(32)
        self.dense2 = MyDense(1)

    def call(self, inputs):
        x = tf.nn.relu(self.dense1(inputs))
        return self.dense2(x)

model = MyModel()
x = tf.random.normal((5, 4))
y = model(x)
print(y)

model = MyModel()
x = tf.random.normal((5, 4))
y = model(x)
print(y)


tf.Tensor(
[[ 0.18456194]
 [ 0.17011364]
 [ 0.23588863]
 [ 0.35789073]
 [-0.05117509]], shape=(5, 1), dtype=float32)
tf.Tensor(
[[0.15793273]
 [0.38999715]
 [0.36287385]
 [0.5769989 ]
 [0.00998762]], shape=(5, 1), dtype=float32)


Custom Loss & Metric

In [7]:
def custom_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

class CustomMAE(tf.keras.metrics.Metric):
    def __init__(self, name="custom_mae"):
        super().__init__(name=name)
        self.total_error = self.add_weight(initializer="zeros")
        self.count = self.add_weight(initializer="zeros")

    def update_state(self, y_true, y_pred):
        error = tf.reduce_sum(tf.abs(y_true - y_pred))
        self.total_error.assign_add(error)
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total_error / self.count

    def reset_state(self):
        self.total_error.assign(0)
        self.count.assign(0)


Automatic Differentiation (GradientTape) & Custom Training Loop

In [8]:
x = tf.random.normal((10, 4))
y_true = tf.random.normal((10, 1))

with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = custom_mse(y_true, y_pred)

grads = tape.gradient(loss, model.trainable_variables)

print("Loss:", loss.numpy())
print("Jumlah gradient:", len(grads))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
metric = CustomMAE()

EPOCHS = 5

for epoch in range(EPOCHS):
    metric.reset_state()

    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = custom_mse(y_true, y_pred)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    metric.update_state(y_true, y_pred)

    print(f"Epoch {epoch+1} | Loss: {loss.numpy():.4f} | MAE: {metric.result().numpy():.4f}")


Loss: 2.5660796
Jumlah gradient: 4
Epoch 1 | Loss: 2.5661 | MAE: 1.4146
Epoch 2 | Loss: 2.2290 | MAE: 1.3315
Epoch 3 | Loss: 1.9485 | MAE: 1.2479
Epoch 4 | Loss: 1.7161 | MAE: 1.1634
Epoch 5 | Loss: 1.5302 | MAE: 1.0795


tf.function (Graph Optimization)

In [10]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = custom_mse(y, y_pred)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

for epoch in range(3):
    loss = train_step(x, y_true)
    print(f"Epoch {epoch+1} | Loss: {loss.numpy():.4f}")

class SimpleSGD(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, name="SimpleSGD"):
        super().__init__(name)
        self.lr = learning_rate

    def apply_gradients(self, grads_and_vars):
        for grad, var in grads_and_vars:
            if grad is not None:
                var.assign_sub(self.lr * grad)


Epoch 1 | Loss: 1.1111
Epoch 2 | Loss: 1.0447
Epoch 3 | Loss: 0.9791


In [16]:
import tensorflow as tf
from keras.optimizers import Optimizer

class SimpleSGD(Optimizer):
    def __init__(self, learning_rate=0.01, name="SimpleSGD", **kwargs):
        super().__init__(
            learning_rate=learning_rate,
            name=name,
            **kwargs
        )

    def update_step(self, grad, variable, learning_rate):
        variable.assign_sub(learning_rate * grad)

    def get_config(self):
        config = super().get_config()
        return config

optimizer = SimpleSGD(learning_rate=0.01)

Saving & Loading Custom Model

In [18]:
class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self.dense1 = MyDense(64, activation="relu")
        self.dense2 = MyDense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)


In [19]:
    def get_config(self):
        config = super().get_config()
        return config

In [20]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs):
        z = tf.matmul(inputs, self.w) + self.b
        return self.activation(z) if self.activation else z

    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "activation": tf.keras.activations.serialize(self.activation)
        })
        return config


In [24]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="kernel"
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )

    def call(self, inputs):
        z = tf.matmul(inputs, self.w) + self.b
        return self.activation(z) if self.activation else z

    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "activation": tf.keras.activations.serialize(self.activation)
        })
        return config

class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = MyDense(64, activation="relu")
        self.dense2 = MyDense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

    def get_config(self):
        return super().get_config()

INPUT_DIM = 3
X = np.random.rand(100, INPUT_DIM).astype(np.float32)
y = np.sum(X, axis=1, keepdims=True)

model = MyModel()
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=custom_mse,
    metrics=["mae"]
)

model(tf.zeros((1, INPUT_DIM)))

model.fit(X, y, epochs=3, verbose=1)

model.save("custom_model.keras")
print("✅ Model saved")

loaded_model = tf.keras.models.load_model(
    "custom_model.keras",
    custom_objects={
        "MyModel": MyModel,
        "MyDense": MyDense,
        "custom_mse": custom_mse
    }
)

print("✅ Model loaded successfully")

pred = loaded_model.predict(X[:5])
print("Prediction:", pred)



Epoch 1/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.2548 - mae: 1.0293  
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0845 - mae: 0.2292 
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2036 - mae: 0.4446 
✅ Model saved
✅ Model loaded successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Prediction: [[1.9437401]
 [1.7235119]
 [2.8013303]
 [2.2792397]
 [1.3715988]]
